In [1]:
from tilepack.builder import build_tile_packages
from configparser import ConfigParser
import os
import shutil
import secrets as secrets
from glob import glob, iglob
from pathlib import Path
import zipfile
import json

os.environ["MAPZEN_API_KEY"] = secrets.MAPZEN_API_KEY

# Download MBTiles & GeoJSON by script

In [2]:
cfg: ConfigParser = ConfigParser()
cfg.read('../config/config.cfg')
cfg_dict: dict = cfg._sections['mbtiles_downloader']
cfg_dict = {
    'min_lon': float(cfg_dict['min_lon']),
    'min_lat': float(cfg_dict['min_lat']),
    'max_lon': float(cfg_dict['max_lon']),
    'max_lat': float(cfg_dict['max_lat']),
    'min_zoom': int(cfg_dict['min_zoom']),
    'max_zoom': int(cfg_dict['max_zoom']),
    'output': cfg_dict['output']
}
cfg_dict

{'min_lon': 14.397,
 'min_lat': 50.1133,
 'max_lon': 14.4137,
 'max_lat': 50.1224,
 'min_zoom': 16,
 'max_zoom': 17,
 'output': 'zoo_prague_map'}

In [3]:
mapzen_url_prefix = os.getenv('MAPZEN_URL_PREFIX', 'https://tile.nextzen.org/tilezen')
mapzen_api_key = os.getenv('MAPZEN_API_KEY', None)
if(mapzen_api_key is None):
    raise RuntimeError('Environmental variable MAPZEN_API_KEY is not set.')

args = {
    'tile_size': 512,
    'tile_format': 'json',
    'output_formats': ['mbtiles', 'zipfile'],
    'layer': 'all',
    'type': 'vector',
    'tile_compression': False,
    'concurrency': 1,
    'api_key': mapzen_api_key,
    'url_prefix': mapzen_url_prefix
}
args |= cfg_dict
print(args)

folder_path = Path(args['output'])
geojson_path = folder_path / 'geojsons'

# Get MBTiles & GeoJSON
if(not geojson_path.is_dir()):
    build_tile_packages(**args)
    
    # Move created files into a folder
    if os.path.isdir(folder_path):
        shutil.rmtree(folder_path)

    os.makedirs(folder_path)
    for file in glob(f'{folder_path}*.*'):
        shutil.move(file, folder_path)

    # Unzip GeoJSON files
    with zipfile.ZipFile(folder_path / f'{folder_path}.zip', 'r') as zip_ref:
        zip_ref.extractall(geojson_path)

{'tile_size': 512, 'tile_format': 'json', 'output_formats': ['mbtiles', 'zipfile'], 'layer': 'all', 'type': 'vector', 'tile_compression': False, 'concurrency': 1, 'api_key': 'B8mtKIzWQ7e7Fg22o4LBhw', 'url_prefix': 'https://tile.nextzen.org/tilezen', 'min_lon': 14.397, 'min_lat': 50.1133, 'max_lon': 14.4137, 'max_lat': 50.1224, 'min_zoom': 16, 'max_zoom': 17, 'output': 'zoo_prague_map'}


OperationalError: table tiles already exists

## Parse GeoJSONs

In [ ]:
# Get animal data from GeoJSONs
for geojson in iglob(str(geojson_path / 'all' / '**/*.json'), recursive=True):
    with open(geojson) as f:
        data = json.load(f)
        for poi in data['pois']['features']:
            poi = poi['properties']
            if(poi['kind'] == 'animal'):
                res: dict = {
                    '_id': poi['id'],
                    'name': poi['name']
                }
                print(res)

In [ ]:
# Get animal data from GeoJSONs
res = dict()
for geojson in iglob(str(geojson_path / 'all' / '**/*.json'), recursive=True):
    with open(geojson) as f:
        data = json.load(f)
        for poi in data['buildings']['features']:
            poi = poi['properties']
            if(poi['kind'] == 'building' and poi.get("id") is not None and poi.get("name") is not None):
                res[poi['id']] = poi['name']
                
print(f'There are {len(res)} buildings there.')

for id_, name in res.items():
    print(f'{id_}: {name}')